In [1]:
# Notebook imports, do not care about them!
import jdc

# Introdução

Redes Neurais de Múltiplas Camadas (RNM), ou Multilayer Perceptron (MLP), são modelos computacionais inspirados no sistema nervoso humano, compostas por estruturas matemáticas que simulam os neurônios e suas conexões. Os usos mais comuns desses modelos consistem em tarefas de classificação e regressão, 
o que favorece a aplicação em diversas áreas de pesquisa, sendo uma delas
a de Visão Computacional.

Para compreender as RNM, é importante primeiramente conhecer dois de seus aspectos fundamentais: os **neurônios** e a **arquitetura em camadas**.

## Neurônios
Um neurônio é uma estrutura que aceita como argumento um vetor $\mathbf{x} = \langle x_1, x_2, \ldots, x_n \rangle \in \mathbb{R}^n$
de entrada e **responde com um valor real** de saída. Cada neurônio está relacionado a um vetor de pesos $\mathbf{w} = \langle w_1, w_2, \ldots, w_n \rangle \in \mathbb{R}^n$, de forma que cada componente $x_i$ da entrada é associada ao peso $w_i$. Ao receber a entrada,
o neurônio responde com o valor $f(\mathbf{x}\cdot\mathbf{w} + b)$, onde $f$ é chamada de 
**função de ativação**; $\mathbf{x}\cdot\mathbf{w} = \sum_i x_i \cdot w_i$, ou seja, 
o produto interno entre $\mathbf{x}$ e $\mathbf{w}$, também chamado de $net$ do neurônio; e $b$ é 
uma constante real chamada *bias*, cuja principal função é permitir um ajuste fino 
do valor de saída por meio de um deslocamento horizontal da função de ativação.

Uma função comum de ativação é a **sigmoide**, expressa pela equação:
$$f(x) = \frac{1}{1 + e^{-x}}.$$

Ela possui a importante propriedade de ser contínua e derivável em $(-\infty, +\infty)$,
o que favorece o algoritmo de treinamento a ser em breve explicado, apesar de existirem algumas desvantagens em sua utilização.


## Arquitetura em camadas

Uma RNM é organizada em uma sequência de $k > 2$ camadas de neurônios, denotadas aqui por $L_0, L_1, \ldots, L_{k-1}$. O tamanho de uma camada $L_i$, denotado aqui por $tam\;L_i$, é a quantidade de neurônios que ela possui. A camada $L_0$ é dita de entrada, na qual existe um **neurônio de entrada** para cada componente do vetor de entrada da rede. Esses neurônios se diferenciam dos demais porque,
ao invés de aceitarem todo o vetor, recebem apenas uma componente e simplesmente a **transmitem** para o interior da rede, sem 
a aplicação da função de ativação. Já a camada $L_{k-1}$ é dita de saída,
e a sua resposta é a resposta da rede.

O que viabiliza o funcionamento do modelo são as conexões entre essas camadas, e elas podem ocorrer de diversas maneiras na rede. Aqui,  
o objeto de estudo são as **redes densas *feedforward***, em que cada neurônio da camada $L_i$ se conecta com
todos os neurônios da camada $L_{i+1}$, $i = 0,\ldots, k-2$, de maneira que o vetor de entrada dos neurônios da camada $L_{i+1}$ é o vetor composto pelas
respostas dos neurônios da camada $L_{i}$. Assim, um neurônio na camada $L_{i+1}$ recebe, como entrada, um vetor
de dimensão $tam\;L_i$, $i=0,1, \ldots, k-2$.

Cada conexão entre neurônios pode ser vista como associada ao peso que o neurônio
atribui ao valor por ela provido. Essa visão possibilita a utilização de uma
notação muito útil para se trabalhar com pesos: denota-se por $w_{ij}^l$ o peso 
da conexão entre o neurônio $j$ da camada $l-1$ com o neurônio $i$ da camada $l$,
$l = 1, \ldots, k-1$. Mais ainda, é possível representar todas as conexões entre
duas camadas por uma matriz de pesos $\mathbf{W}^l = (w_{ij}^l)$ de dimensão $tam\;L_{l} \times tam\;L_{l-1}$.
Para facilitar a identificação dos *biases* de cada neurônio em uma camada, denota-se por 
$\mathbf{b}^l = \langle b^l_1, b^l_2, \ldots, b^l_{tam\;L_l} \rangle$ os *biases* 
dos neurônios da camada $L_l$, sendo $b^l_i$ o *bias* do neurônio $i$ na camada $L_l$.

Sabendo disso, já é possível iniciar a implementação em Python da RNM!

### Implementando a classe `MultilayerNeuralNetwork`

Como o objetivo é
produzir um módulo reutilizável, os recursos de orientação a objetos da linguagem Python
serão aplicados. Além disso, o pacote `numpy` será utilizado para a manipulação de matrizes. 

A programação tem início com a importação do `numpy` e a declaração da classe `MultilayerNeuralNetwork`, a qual encapsulará todos os dados e métodos necessários ao
funcionamento e à utilização da rede, como a matriz de pesos, a arquitetura e as rotinas de treinamento:

In [7]:
import numpy as np

class MultilayerNeuralNetwork:
    '''A Multilayer Neural Network implementation.'''

Essa é apenas uma classe, e nada tem a ver com redes neurais além do nome simplesmente. O próximo passo é criar um **construtor**, método que sempre será chamado quando alguém
criar (ou, mais tecnicamente, instanciar) a rede neural. Nele, é importante garantir
a criação e inicialização da arquitetura, o que implica na configuração das camadas (quantas e quantos neurônios devem possuir), das matrizes de pesos e dos vetores de *bias*.  Estas serão inicializadas com valores aleatórios, seguindo
a distribuição normal, e, em seguida, normalizadas pela raiz quadrada da quantidade de neurônios da camada correspondente. A inicialização dos pesos é um tópico importante, para o qual existem diversas propostas, mas que será abstraído neste momento por economia de tempo.
Além disso, um parâmetro `alpha` será acrescentado, 
cuja existência será justificada mais adiante. Portanto, o construtor receberá do
usuário a **arquitetura da rede**, no formato $[tam\;L_0, tam\;L_1, \ldots, tam\;L_{k-1}]$, e o tal `alpha`:

In [14]:
    %%add_to MultilayerNeuralNetwork
    def __init__(self, arch = [1,2,1], alpha = 0.1):
        '''Initialize the network.'''
        
        self.W = []
        self.B = []
        self.alpha = alpha
        self.arch = arch
        
        # Initialize the weight matrix and biases with normalized random values
        for i in np.arange(0,len(self._arch)-1):
            # Weight
            w = np.random.randn(self._arch[i], self._arch[i+1])
            self.W.append(w/np.sqrt(self._arch[i]))
            # Bias
            b = np.random.randn(self._arch[i],1)
            self.B.append(b/np.sqrt(self._arch[i]))

Como será utilizada a função sigmoide para a ativação dos neurônios, é importante
que ela e sua derivada estejam implementadas na classe:  

In [15]:
    %%add_to MultilayerNeuralNetwork
    def sigmoid(self, x):
        '''Sigmoid function.'''
        return 1.0/(1 + np.exp(-x))
    
    def sigmoid_deriv(self, x):
        '''Derivative of the sigmoid function, considering that x is the result
        of applying the sigmoid function to the net.
        '''
        return x * (1 - x)

Com isso, é possível instanciar uma rede, mesmo que não se possa fazer nada com ela ainda, apenas para checar a sua estrutura:

In [13]:
# Criando uma arquitetura com 2 neurônios na camada de entrada, uma camada escondida com 3 neurônios e 1 neurônio
# na camada de saída.

neuralnet = MultilayerNeuralNetwork(arch = [2,3,1], alpha = 0.5)
print(neuralnet.W)

[array([[-0.35191368,  0.11590091, -0.10216289],
       [ 1.452429  , -0.44705065, -0.63183561]]), array([[ 1.02146438],
       [-0.02718928],
       [ 0.41438558]])]


### Respostas das camadas

Note que cada camada produz um vetor de resposta composto pelas respostas individuais
de cada um de seus neurônios. Denote por $\mathbf{a^l} = \langle a^l_1, a^l_2, \ldots, a^l_{tam\; L_l} \rangle$ esse vetor para a camada $L_l$. Como visto, ele deverá ser a entrada para a camada seguinte, mas os seus valores serão combinados de forma particular
pela matriz de pesos $\mathbf{W}^{l+1}$. A expressão

$$\mathbf{z}^l = \mathbf{W}^{l+1}\cdot\mathbf{a^l} + \mathbf{b}^l$$

produz um vetor cujos componentes são os valores $net$ de cada neurônio da camada
$L_{l+1}$. Sabendo disso, qual seria a expressão para o vetor de saída da camada $L_{l+1}$, ou seja, para $\mathbf{a}^{l+1}$?
Basta aplicar a função de ativação a cada componente do vetor obtido! Considerando-se a aplicação da função ponto a ponto, ou seja, $f([x_1, x_2, \ldots, x_n]) = [f(x_1), f(x_2), \ldots, f(x_n)]$, tem-se que:

$$\mathbf{a}^{l+1} = f(\mathbf{W}^{l+1}\cdot\mathbf{a^l} + \mathbf{b}^l) = f(\mathbf{z}^l).$$

O conhecimento adquirido até este ponto é suficiente para se entender a arquitetura de uma rede neural e como computar as saídas das suas camadas de neurônios a partir 
de um vetor de entrada. Ocorre que as matrizes de pesos aleatórios 
tornam o modelo inútil. Ele necessita se ajustar (lê-se "aprender")
para solucionar os problemas com os quais é confrontado, e é disso que 
trataremos a partir de agora!

# Aprendizado em uma RNM

O modelo de aprendizagem de redes neurais é geralmente o **supervisionado**, pois faz
uso de exemplos representantes da verdade (*ground truth*) para fazer
com que a rede gere as respostas desejadas após uma **etapa de treinamento**. É como se um professor
apresentasse a entrada, a rede respondesse e ele informasse qual foi o 
erro cometido. A rede, com base nisso, modifica sua estrutura (suas matrizes
 de pesos) para que o erro, da próxima vez em que o professor mostre
 o exemplo, seja garantidamente menor. Esse processo termina quando algum critério de parada é atingido. Os mais comuns são o erro máximo e o número de ciclos ou *epochs*.
 
 O erro cometido pela rede é modelado matematicamente por uma **função de custo**
 $C:\mathbb{R}^n \to \mathbb{R}$. As funções que podem desempenhar esse papel devem cumprir alguns requisitos
 básicos. Uma das mais comuns é a do **erro quadrático**, de equação:
 
 $$C_x = \frac 1 2 ||\mathbf{y}(\mathbf{x})- a^n(\mathbf{x})||^2 = \frac 1 2 \sum_i (y_i(\mathbf{x}) - a^n_i(\mathbf{x}))^2,$$
 onde $\mathbf{y}(\mathbf{x})$ é a saída esperada para o exemplo $\mathbf{x}$ e $a^n(\mathbf{x})$ é a saída da última camada da rede para o exemplo $\mathbf{x}$. Note que
 quanto maior a diferença entre a verdade e a saída da rede, maior o valor dessa
 função. 
 
 Assim, o objetivo do treinamento é fazer com que o valor de $C$, para cada exemplo,
 seja menor a cada ciclo. Como $C$ é função dos pesos, nada melhor que buscar ajustá-los
 a fim de alcançar esse objetivo.
 
 Esse ajuste, entretanto, demanda o conhecimento sobre como variações nas matrizes
 de pesos - de camadas diferentes da última, inclusive - afetam o resultado da função de custo. Em outras palavras, 
 é interessante conhecer 
 
 $$\frac{\partial C}{\partial w_{ij}^l}.$$
 
 Para tanto, existe um caminho implícito. Imagine que o neurônio $j$ da camada $L_l$
 deve produzir o valor $net$ denotado por $z_j^l$. Porém, suponha que esse valor
 seja alterado por uma quantidade pequena $\Delta z_j^l$, de tal forma que a resposta dele
 seja $f(z_j^l + \Delta z_j^l)$. Como a função de custo é afetada por essa mudança?
 Do polinômio de Taylor para o Cálculo Multivariável, chega-se que 
 
 $$\Delta C = \frac{\partial C}{\partial z_{j}^l} \Delta z_{j}^l.$$
 
 Note que, se $\frac{\partial C}{\partial z_{j}^l}$ 
 possuir um valor alto, é necessário um $\Delta z_{j}^l$ de sinal oposto
 para reduzir o custo. Se possuir um valor próximo de zero, 
 como $\Delta z_j^l$ é pequeno, não se pode fazer muito para reduzir o custo, e o neurônio é dito estar
 em estado próximo do ótimo. Assim, faz sentido definir o erro nesse neurônio, 
 denotado por $\delta_j^l$, como
 
 \begin{equation}
 \delta_j^l = \frac{\partial C}{\partial z_{j}^l}.
 \end{equation}
 
 Além disso, define-se o vetor de erros dos neurônios da camada $L_l$ por
 $\delta^l = \langle \delta^l_1, \delta^l_2, \ldots, \delta^l_{tam\;L_l}  \rangle$.
 O algoritmo explicado na seção seguinte, denominado *Backpropagation*, provê meios de calcular os erros como definidos para se chegar às derivadas parciais $\frac{\partial C}{\partial w^l_j}$ em todas as camadas. Depois, uma regra de otimização (neste estudo, a regra do **gradiente descendente**) as utilizará para atualizar os pesos de acordo, permitindo o aprendizado da rede.

## O algoritmo Backpropagation

Pode-se dizer que este é o grande viabilizador das redes neurais modernas, incluindo
as utilizadas para *Deep Learning*. Este algoritmo utiliza as definições anteriores e está fundamentado em três equações, explicadas a seguir. Para simplificar a notação, considere a camada $L_{k-1}$ (de saída) denotada pelo índice $L$.

### Uma equação para o $\delta^L$

Esta equação fornece meios para se calcular o erro na camada de saída. Sabemos que a função de custo tem 
a forma $C=C(X_1, X_2, \ldots, X_{tam\,L})$, tal que $X_1 = a_1^L(z^L_1, z^L_2 \ldots, z^L_{tam\,L}), X_2 = a_2^L(z^L_1, z^L_2 \ldots, z^L_{tam\,L}), \ldots, X_{tam\,L} = a_{tam\, L}^L(z^L_1, z^L_2 \ldots, z^L_{tam\,L})$. Pela definição anterior, sabemos que:

$$\delta_j^L = \frac{\partial C}{\partial z_{j}^L}.$$

Utilizando a **regra da cadeia** do Cálculo Multivariável, tem-se que

$$\frac{\partial C}{\partial z_{j}^L} = \sum_k \frac{\partial C}{\partial a^L_j}\frac{\partial a^L_k}{\partial z^L_j}.$$

Como apenas $a^L_j$ depende de $z^L_j$, todas as $\frac{\partial a^L_k}{\partial z^L_j}$, com $k \neq j$, serão anuladas, restando que:

$$\frac{\partial C}{\partial z_{j}^L} =\frac{\partial C}{\partial a^L_j}\frac{\partial a^L_j}{\partial z^L_j}.$$

Como $a^L_j = \sigma(z^L_j)$, 

$$\frac{\partial a^L_j}{\partial z^L_j} = \sigma'(z^L_j).$$

Finalmente, chega-se à equação

$$\delta^L_j = \frac{\partial C}{\partial a^L_j}\sigma'(z_j^L).$$

Na forma vetorial, temos:

$$\delta^L = \nabla C_a \odot \sigma'(\mathbf{z}^L).$$

### Uma equação para o $\delta^l$ em função de $\delta^{l+1}$

Na equação passada, conseguimos uma forma de calcular os erros da última camada, entretanto, queremos também um modo
de se calcularem os erros nas demais camadas. A estratégia é tentar escrever $\delta^{l}_j = \frac{\partial C}{\partial z^l_j}$ em termos de $\delta^{l+1}_{j} = \frac{\partial C}{\partial z^{l+1}_j}$. Como? Primeiro, é válido notar que $z_j^{l+1}$, por definição,
depende $z_j^l$:

$$z_j^{l+1} = \mathbf{W}^{l+1}_ja^l = \mathbf{W}^{l+1}_jf(z^l) = \sum_i w^{l+1}_{ij}f(z^l_i).$$

Além disso, $C$ depende de $z_j^{l+1}$, devido à definição recursiva de $a^n$. Com isso, basta aplicar a regra da cadeia novamente, e temos que:

$$\delta_j^l = \sum_k \frac{\partial C}{\partial z^{l+1}_k}\frac{\partial z^{l+1}_k}{\partial z^l_j} = \sum_k \frac{\partial z^{l+1}_k}{\partial z^l_k}\delta^{l+1}_k.$$

Diferenciando a expressão para $z_j^{l+1}$, temos que:
$$\frac{\partial z_k^{l+1}}{\partial z_j^l} = w^{l+1}_{jk}f'(z^l_j).$$

Assim, substituindo, chegamos a:
$$\delta_j^l = \sum_k w^{l+1}_{jk}\delta^{l+1}_kf'(z^l_j).$$

### Uma equação para $\frac{\partial C}{\partial w^l_{jk}}$

Sabemos que $C$ está em função de $z^l$ e que $z^l$ está em função dos pesos,
o que inclui $w_{jk}^l$. Por essa razão, podemos novamente aplicar a regra
da cadeia, da seguinte forma:
$$\frac{\partial C}{\partial w^l_{jk}}=\sum_m \frac{\partial C}{\partial z_m^l} \frac{\partial z_m^l}{\partial w_{jk}^l}$$

Note que $w^l_{jk}$ apenas influencia no cálculo de $z^l_j$, logo
$$\frac{\partial C}{\partial w^l_{jk}}=\frac{\partial C}{\partial z_j^l} \frac{\partial z_j^l}{\partial w_{jk}^l}=\delta^l_j\frac{\partial z_j^l}{\partial w_{jk}^l}.$$

Lembremo-nos de que:
$$z_j^{l} = W^{l}_ja^{l-1} = \sum_i w^{l}_{ij}a^{l-1}_i.$$

Com isso, diferenciando, temos:
$$\frac{\partial z_j^{l}}{\partial w_{jk}^l} =  a^{l-1}_j.$$

Finalmente, substituindo:
$$\frac{\partial C}{\partial w^l_{jk}}=\delta^l_j a^{l-1}_j.$$


Temos, assim, todas as equações fundamentais que constituem o algoritmo
Backpropagation. Agora, resta implementarmos. Primeiramente, 
precisamos de um método que represente todo o processo de treinamento:

In [16]:
    %%add_to MultilayerNeuralNetwork
    def fit(self, X, y, epochs = 1000, displayUpdate = 100):
        '''Fit the model to the training data.'''
        # First, insert the columns of 1's, to perform the bias trick
        #X = np.c_[X, np.ones((X.shape[0]))]
        
        # Loop through the number of epochs
        for ep in np.arange(0, epochs):
            # Fit each training point
            for (x, target) in zip(X, y):
                self.fit_partial(x, target)
            # Print an update
            if ep == 0 or (ep+1) % displayUpdate == 0:
                loss = self.calculate_loss(X, y)
                print("[INFO] Epoch: {}, Loss: {:.7f}".format(ep + 1, loss))

Note que devemos definir duas funções: `fit_partial` e `calculate_loss`. A primeira
é o coração do algoritmo Backpropagation e segue abaixo implementada:

In [17]:
    %%add_to MultilayerNeuralNetwork
    def fit_partial(self, x, y):
        # We start creating the matrix which will store the activation values, a^l.
        # For the first layer, it just takes the input:
        A = [np.atleast_2d(x)]
        # FEEDFORWARD: compute all the outputs from all the neurons, storing in A
        for l in np.arange(0,len(self.W)):
            net = A[l].dot(self.W[l]) + B[l]
            out = self.sigmoid(net)
            A.append(out)
        # BACKPROPAGATION
        # Using Equation 1 to compute delta^L:
        error = (A[-1] - y)
        D = [error * self.sigmoid_deriv(A[-1])]
        
        for layer in np.arange(len(A)-2, 0, -1):
            # Using Equation 2 to compute all deltas
            delta = D[-1].dot(self.W[layer].T) 
            delta = delta * self.sigmoid_deriv(A[layer])
            D.append(delta)
        # Reverse the deltas
        D = D[::-1]
                
        # WEIGHT UPDATE: using Equation 3
        for layer in np.arange(0, len(self.W)):
            self.W[layer] += -self.alpha * A[layer].T.dot(D[layer])
        

A segunda função é responsável por computar a função de custo. Nesta implementação, a `calculate_loss` tomará um conjunto de pontos e computará o custo para ele. Vale lembrar, porém, que isso envolve computar a saída da rede para uma dada entrada. Essa tarefa será executada pelo método `predict`, que recebe esse nome por ser utilizado nas predições realizadas no uso efetivo da rede, após o treinamento.

Seguem as implementações dos métodos `predict` e `calculate_loss`:

In [ ]:
    %%add_to MultilayerNeuralNetwork
    def predict(self, X, addBias = True):
        p = np.atleast_2d(X)
        if addBias:
            p = np.c_[p, np.ones((p.shape[0]))]
        for layer in np.arange(0, len(self.W)):
            p = self.sigmoid(np.dot(p, self.W[layer]))
        return p
    
    def calculate_loss(self, X, targets):
        targets = np.atleast_2d(targets)
        predictions = self.predict(X, addBias = False)
        loss = 0.5 * np.sum((predictions - targets) ** 2)
        return loss
        

# Utilizando a rede neural

Agora que temos nossa rede neural implementada, podemos resolver alguns problemas bastante interessantes. Aqui serão mostrados dois exemplos: o XOR e o reconhecimento de caracteres utilizando o *dataset* MNIST.

## XOR

XOR é uma operação bem conhecida da lógica proposicional que possui a seguinte tabela verdade:

|$x_1$|$x_2$|$y$
|---|---|---|
|0|0|0|
|1|0|1|
|0|1|1|
|1|1|0|

A partir disso, tem-se o *dataset* XOR, composto pelos vetores:
$\langle 0,0 \rangle, \langle 1,0 \rangle, \langle 0,1 \rangle, \langle 1,1 \rangle$,
com os respectivos *labels* $0, 1 , 1, 0$, correspondendo a cada linha da tabela acima. O que acontece se plotarmos 
$x_1$ no eixo $x$, $x_2$ no eixo $y$ e atribuirmos uma cor para cada um dos dois valores possíveis para $y$? Note:


In [ ]:
import matplotlib.pyplot as plt

x1 = np.array([[0],[1],[0],[1]])
x2 = np.array([[0],[0],[1],[1]])

plt.scatter(x1,x2,c=['green','red','red','green'])
plt.title('The XOR dataset')
plt.show()

O que há de importante nesse *dataset*? **Não é possível separá-lo utilizando uma reta**! Ou seja, não corresponde a um problema de classificação com classes linearmente separáveis. Um neurônio somente não conseguiria resolvê-lo, mas nossa rede pode! Vamos?

In [ ]:
# Vamos compor nosso dataset X justapondo horizontalmente x1 e x2 declarados anteriormente
X = np.concatenate((x1,x2), axis=1)
# Temos também de declarar os targets
y = np.array([[0],[1],[1],[0]])

# Agora, instanciamos e treinamos nossa rede neural!
xorNetwork = MultilayerNeuralNetwork(arch=[2,2,1], alpha=0.5)
xorNetwork.fit(X, y, epochs = 20000, displayUpdate=100)

## MNIST

O MNIST é um *dataset* massivamente utilizado pelos estudiosos de Machine Learning composto
por imagens de dígitos de 0 a 9. Ele possui 60000 exemplos de treino e 10000 de testes. 
Os vetores de características são 784-dimensionais ($28 \times 28$ pixels por imagem),
com componentes assumindo valores em $[0,255]$. O propósito é corretamente classificar
os dígitos desse *dataset*!

A biblioteca `sklearn` oferece o uma amostra do MNIST por meio de comandos simples. 
Vamos utilizá-los para obter esse *dataset* e, com ele,
treinar uma rede neural especializada em classificar
seus dígitos de 0 a 9!


In [ ]:
# Import sklearn tools
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets

# Load a sample of MNIST through simple sklearn commands
print("[INFO] loading MNIST dataset")
digits = datasets.load_digits()
data = digits.data.astype("float")
data = (data - data.min()) / (data.max() - data.min())    # normalize data
print("[INFO] samples: {}, dim: {}".format(data.shape[0], data.shape[1]))

# Split the dataset into train and test
(trainX, testX, trainY, testY) = train_test_split(data, digits.target, test_size=0.25)

# Binarize labels
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

# Train the neural network
print("[INFO] training network...")
mnistNN = MultilayerNeuralNetwork(arch=[trainX.shape[1], 32, 16, 10], alpha=0.1)
mnistNN.fit(trainX, trainY, epochs=1000)

Uma vez treinada a rede, podemos checar seu desempenho no conjunto de testes, por meio da função `classification_report`, que fornece índices de desempenho e uma **matriz de confusão**:

In [ ]:
print("[INFO] evaluating network...")
# Test the model
predictions = mnistNN.predict(testX)
predictions = predictions.argmax(axis=1)
print(classification_report(testY.argmax(axis=1), predictions))

# Redes Neurais com Keras

O que fizemos até então geralmente não é feito na prática. Existem bibliotecas mais robustas e otimizadas para se trabalhar com redes neurais e outros modelos. O Keras é uma biblioteca para Python muito utilizada para Machine Learning e permite o treino e avaliação de redes neurais com poucas linhas de código.

In [ ]:
# Import sklearn and keras tools
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import datasets
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
import numpy as np

# Download the full MNIST dataset
dataset = datasets.fetch_mldata("MNIST Original")

# Normalize data
data = dataset.data.astype("float")/255.0

# Split data into sets
(trainX, testX, trainY, testY) = train_test_split(data, dataset.target, test_size=0.25)

# Binarize labels
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

# Prepare the feedforward neural network with keras
model = Sequential()
model.add(Dense(256, input_shape=(784,), activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(10, activation="softmax"))

# Train the neural network
print("[INFO] training...")
sgd = SGD(0.1)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
H = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=128)